In [1]:
import numpy as np
import pandas as pd

In [2]:
def read_command(path):
    with open(path) as f:
        lines = f.readlines()
        
    com = []    
    count = 0
    for l in lines:
        if count == 0 and '# Example' in l:
            count += 1
        elif count == 1 and l == '#\n':
            count += 1
        elif count == 2 and '########################' in l:
            count += 1
        elif count ==3 and l == '\n':
            count += 1
        elif count == 4:
            com.append(l.strip())
            count = 0
            
    return com

path = './EGPSR Examples.txt'
com_lst = read_command(path)  
print('len: ',len(com_lst))
com_lst[:10]

len:  10000


['Please spot -B_find- a orange -B_find_obj- in -I_find_obj- the -I_find_obj- living -i- room -I_find_obj-, take -B_take- the paprika -B_take_obj-, and tell -B_say- the time -I_say-',
 'Put -B_take- the orange -B_take_obj- into the basket -i- on -i- the -i- storage -i- table -end- -B_take_dest- please',
 'Could you please tell -B_say- me how many men in the corridor are sitting -I_say-',
 'Pick -B_take- up -I_take- a cutlery -B_take_obj- from the dining -i- table -B_take_sour- and put -B_take- it -B_take_obj- into the basket -i- on -i- the -i- counter -end- -B_take_dest-',
 'Grasp -B_take- the fruits -B_take_obj- from the dining -i- table -B_take_sour-, put -B_take- it -B_take_obj- on the counter -B_take_dest-, and place -B_take- paprika -B_take_obj- on the end -i- table -B_take_dest-',
 'Look -B_find- for -I_find- Francis -B_find_per- at -I_find_per- the -I_find_per- couch -I_find_per-, pinpoint -B_find- Jennifer -B_find_per- in -I_find_per- the -I_find_per- living -i- room -I_find_pe

In [3]:
import itertools

def improve_labels(com_lst):
    res = []
    for i,com in enumerate(com_lst):
        com = com.split()
        if '-B_say-' not in com and '-i-' not in com:
            res.append(com)
            continue
        elif '-B_say-' in com:
            if '-I_say-' not in com:
                labelled = com
            else:
                b_idx = com.index('-B_say-') + 1
                e_idx = com.index('-I_say-') -1
                first = com[:b_idx]
                end = com[e_idx:]
                
                need_label = com[b_idx:e_idx]
                
                #rem all -i- in the 'say' cases
                need_label = [ele for ele in need_label if ele != '-i-']
                
                interleave = list(itertools.chain(*zip(need_label, ['-I_say-']*len(need_label))))
                labelled = first + interleave + end
                
                com = labelled.copy()
                
        elif '-i-' in com:       
            string = ' '.join(com)
            while '-i- -i-' in string:
                string = string.replace('-i- -i-', '-i-')
            com = string.split()
            if i == 13:
                print(com)
                
            idx_lst = []
            for idx,ele in enumerate(com):
                if ele == '-i-':
                    if len(idx_lst) == 0: 
                        idx_lst.append([idx])
                    elif idx == idx_lst[-1][-1] + 2:
                        idx_lst[-1].append(idx)
                    else:
                        idx_lst.append([idx])
                        
            for lst in idx_lst:
                end_label = com[lst[-1]+2]
                com[lst[0]] = end_label
                if 'B' in end_label:
                    com[lst[-1]+2] = com[lst[-1]+2].replace('B','I')
                    
                for i,idx in enumerate(lst):
                    if i == 0:
                        continue
                    else:
                        label = com[lst[-1]+2]
                        com[idx] = label
                        
            labelled = com.copy()
               
                
        res.append(labelled)     
    return res

res = improve_labels(com_lst)

['Pick', '-B_take-', 'up', '-I_take-', 'the', 'orange', '-B_take_obj-', 'from', 'the', 'storage', '-i-', 'table', '-B_take_sour-,', 'deliver', '-B_take-', 'to', 'it', '-B_take_obj-', 'to', 'Robin', '-B_take_dest-', 'at', '-I_take_dest-', 'the', '-I_take_dest-', 'entrance', '-I_take_dest-,', 'locate', '-B_find-', 'the', 'cascade', '-i-', 'pod', '-B_find_obj-']


In [5]:
#res[3]
res[21]

['Tell',
 '-B_say-',
 'me',
 '-I_say-',
 'how',
 '-I_say-',
 'many',
 '-I_say-',
 'children',
 '-I_say-',
 'are',
 '-I_say-',
 'wearing',
 '-I_say-',
 'T-shirts',
 '-I_say-',
 'in',
 '-I_say-',
 'the',
 '-I_say-',
 'corridor',
 '-I_say-',
 'please']

# Generate Seq Out ( Copied from mbot, but changed intro to emplty lst)

In [12]:
def generate_seq_out(tasks):
    c = 0
    sentences = []
    outputs = []
    for v in range(len(tasks)):

#         try:
#             task = tasks[v].split(' ')
#         except:
#             print('error at {}, check if the items in tasks are strings tasks[v] = {}'.format(v, tasks[v]))
#             break
        task = tasks[v]
        # intro = intros[c].split(' ')
        intros = []
        intro = []

        sentence = []
        output = []
        # appending intro if the total length is less than or equal to 15 (ref: Pedro master thesis) ############################################################
        task_with_intro = []
        if v%4 == 0 and (len(intro) + len(task)) <= 15:
            for x in intro:
                task_with_intro.append(x)
            c = c + 1

        # reinitiating intros
        if c == len(intros):
            c = 0

        # appending task
        for x in task:
            task_with_intro.append(x)

        # appending the task with introduction to the sentence list
        for h in range(len(task_with_intro)):
            if not task_with_intro[h].startswith('-'):
                sentence.append(task_with_intro[h])
                # appending outputs according to the input
                if h < len(task_with_intro)-1:
                    if task_with_intro[h+1].startswith('-'):
                        l = task_with_intro[h+1]
                        l = l.replace('-', '')
                        output.append(l)
                    else:
                        output.append('O')
                else:
                    output.append('O')
        # split sentences and tags
        sentences.append(sentence)
        outputs.append(output)
    return outputs

seq_out =  generate_seq_out(res)

In [23]:
seq_in = [[string for string in com.split() if '-' not in string] for com in com_lst]
print(seq_in[20])
seq_out[20]

['Greet', 'the', 'person', 'raising', 'their', 'right', 'arm', 'in', 'the', 'living', 'room', 'please']


['B_greet',
 'O',
 'B_greet_per',
 'I_greet_per',
 'I_greet_per',
 'I_greet_per',
 'I_greet_per',
 'I_greet_per',
 'I_greet_per',
 'I_greet_per',
 'I_greet_per',
 'O']

In [ ]:
v

# Save seq_in and Seq_out

In [27]:
with open('dataset_seq_in.txt', 'w') as f:
    for lst in seq_in:
        com = ' '.join(lst)
        f.write(com)
        f.write('\n')
        
        
with open('dataset_seq_out.txt', 'w') as f:
    for lst in seq_out:
        com = ' '.join(lst)
        f.write(com)
        f.write('\n')

## Generate a readable file

In [31]:
max_seq_len = max(len(x) for x in seq_in)

seq_dict = {(i-1 if i != 0 else 'ID'):[' ']*(len(seq_in)*3) for i in range(max_seq_len + 1)}

for token_idx,t_lst in seq_dict.items():
    for com_idx in range(len(seq_in)):
        df_idx = com_idx*3
        
        if token_idx == 'ID':
            t_lst[df_idx] = com_idx
            t_lst[df_idx+1] = ' '
            t_lst[df_idx+2] = '------------------'
            continue
        
        com_seq_lst = seq_in[com_idx]#dataset_df.loc[com_idx,'Commands'].split()
        max_len = len(com_seq_lst)
        if token_idx > max_len-1:
            continue
        com_token = com_seq_lst[token_idx]
        out_token = seq_out[com_idx]#dataset_df.loc[com_idx,'seq_out'][token_idx]
        t_lst[df_idx] = com_token
        t_lst[df_idx+1] = out_token
        t_lst[df_idx+2] = '------------------'
        seq_dict.update({token_idx:t_lst})
        
    
df_read = pd.DataFrame(seq_dict)
df_read.to_csv('dataset.csv', index = False)
df_read.head(14)

,ID,0,1,2,3,4,5,6,7,8,...,16,17,18,19,20,21,22,23,24,25
0,0,Please,spot,a,orange,in,the,living,room,take,...,,,,,,,,,,
1,,"[O, B_find, O, B_find_obj, I_find_obj, I_find_...","[O, B_find, O, B_find_obj, I_find_obj, I_find_...","[O, B_find, O, B_find_obj, I_find_obj, I_find_...","[O, B_find, O, B_find_obj, I_find_obj, I_find_...","[O, B_find, O, B_find_obj, I_find_obj, I_find_...","[O, B_find, O, B_find_obj, I_find_obj, I_find_...","[O, B_find, O, B_find_obj, I_find_obj, I_find_...","[O, B_find, O, B_find_obj, I_find_obj, I_find_...","[O, B_find, O, B_find_obj, I_find_obj, I_find_...",...,,,,,,,,,,
2,------------------,------------------,------------------,------------------,------------------,------------------,------------------,------------------,------------------,------------------,...,,,,,,,,,,
3,1,Put,the,orange,into,the,basket,on,the,storage,...,,,,,,,,,,
4,,"[B_take, O, B_take_obj, O, O, end, end, end, e...","[B_take, O, B_take_obj, O, O, end, end, end, e...","[B_take, O, B_take_obj, O, O, end, end, end, e...","[B_take, O, B_take_obj, O, O, end, end, end, e...","[B_take, O, B_take_obj, O, O, end, end, end, e...","[B_take, O, B_take_obj, O, O, end, end, end, e...","[B_take, O, B_take_obj, O, O, end, end, end, e...","[B_take, O, B_take_obj, O, O, end, end, end, e...","[B_take, O, B_take_obj, O, O, end, end, end, e...",...,,,,,,,,,,
5,------------------,------------------,------------------,------------------,------------------,------------------,------------------,------------------,------------------,------------------,...,,,,,,,,,,
6,2,Could,you,please,tell,me,how,many,men,in,...,,,,,,,,,,
7,,"[O, O, O, B_say, I_say, I_say, I_say, I_say, I...","[O, O, O, B_say, I_say, I_say, I_say, I_say, I...","[O, O, O, B_say, I_say, I_say, I_say, I_say, I...","[O, O, O, B_say, I_say, I_say, I_say, I_say, I...","[O, O, O, B_say, I_say, I_say, I_say, I_say, I...","[O, O, O, B_say, I_say, I_say, I_say, I_say, I...","[O, O, O, B_say, I_say, I_say, I_say, I_say, I...","[O, O, O, B_say, I_say, I_say, I_say, I_say, I...","[O, O, O, B_say, I_say, I_say, I_say, I_say, I...",...,,,,,,,,,,
8,------------------,------------------,------------------,------------------,------------------,------------------,------------------,------------------,------------------,------------------,...,,,,,,,,,,
9,3,Pick,up,a,cutlery,from,the,dining,table,and,...,counter,,,,,,,,,


In [30]:
seq_dict

{'ID': [' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
